## Get the weather

In [29]:

# 5. Extract `latitude` and `longitude` from `Trip Origin` and `Trip Destination` in `merged_df`
for col in ['Trip Origin', 'Trip Destination']:
    merged_df[[f'{col}_latitude', f'{col}_longitude']] = merged_df[col].str.split(',', expand=True).astype(float)

# 6. Filter to keep only rows where `driver_action` is 'accepted' and create a copy
merged_df_accepted_copy = merged_df[merged_df['driver_action'] == 'accepted'].copy()

# 7. Define a function `get_weather` that takes latitude, longitude, and API key as input and returns the weather description from the OpenWeatherMap API.
def get_weather(lat, lon, api_key):
    base_url = "https://api.openweathermap.org/data/3.0/onecall?"
    complete_url = base_url + "lat=" + str(lat) + "&lon=" + str(lon) + "&appid=" + api_key
    response = requests.get(complete_url)
    if response.status_code == 200:
        data = response.json()
        weather_description = data['current']['weather'][0]['description']
        return weather_description
    else:
        print("Error in API call")
        return None

# 8. Apply the `get_weather` function to the `merged_df_accepted_copy` DataFrame to get the weather conditions for each driver's location.
merged_df_accepted_copy['weather_description'] = merged_df_accepted_copy.apply(
    lambda row: get_weather(row['lat'], row['lng'], api_key), axis=1
)

# 9. Display the first 5 rows of the dataframe with the weather description.
print(merged_df_accepted_copy[['order_id', 'weather_description']].head().to_markdown(index=False, numalign="left", stralign="left"))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x75bb2a7a65e0>>
Traceback (most recent call last):
  File "/home/hilla/code/10Academy-training/week8/Causal-Inference-Delivery-Logistic-Location-Optimization/.venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [28]:
merged_df_accepted_copy.head()

,order_id,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,day_of_week,hour_of_day,day_of_month,month,Trip Origin_latitude,...,is_holiday,is_weekend,Trip Start Date,id,driver_id,driver_action,lat,lng,created_at,updated_at
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01,2021-07-01 09:34:36,Thursday,9,1,July,6.601042,...,0,0,2021-07-01,1.0,243828.0,accepted,6.602207,3.270465,NaN,NaN
14,392005,"6.565087699999999,3.3844415","6.499696300000001,3.3509075",2021-07-01,2021-07-01 11:27:51,Thursday,10,1,July,6.565088,...,0,0,2021-07-01,11.0,245597.0,accepted,6.549147,3.392184,NaN,NaN
65,392009,"6.6636484,3.3082058","6.6185421,3.301634",2021-07-01,2021-07-01 07:41:12,Thursday,6,1,July,6.663648,...,0,0,2021-07-01,62.0,245600.0,accepted,6.644829,3.289328,NaN,NaN
132,392013,"6.4308171,3.4341552","6.435460000000001,3.4846547",2021-07-01,2021-07-01 09:19:11,Thursday,8,1,July,6.430817,...,0,0,2021-07-01,129.0,243892.0,accepted,6.435331,3.424317,NaN,NaN
145,392014,"6.499156300000001,3.3585173","6.4280911,3.5157172",2021-07-01,2021-07-01 07:27:24,Thursday,6,1,July,6.499156,...,0,0,2021-07-01,142.0,243781.0,accepted,6.498221,3.360042,NaN,NaN


In [31]:
df.to_csv('../data/features/merged_df_accepted_copy.csv', index=False)

#### Getting weather info using OPENWEATHERMAP API

In [25]:
# API call to request historical data:

lat, lon = (6.6010417,3.2766339)
time = 1627830000
api_key = os.getenv("OPENWEATHERMAP_API_KEY")

response = requests.get(
            f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={time}&appid={api_key}"
          )

def get_weather(timestamp, lat, lon):
    base_url = "http://api.openweathermap.org/data/2.0/onecall?"
    complete_url = base_url + "lat=" + str(lat) + "&lon=" + str(lon) + "&appid=" + api_key
    response = requests.get(complete_url) 
    x = response.json()
    if x["cod"] != "404":
        return x['weather'][0]['main']  # Simplify to just the main weather condition
    else:
        return np.nan  # Missing if API call fails

In [27]:
response.json()

{'lat': 6.601,
 'lon': 3.2766,
 'timezone': 'Africa/Lagos',
 'timezone_offset': 3600,
 'data': [{'dt': 1627830000,
   'sunrise': 1627796462,
   'sunset': 1627841116,
   'temp': 301.54,
   'feels_like': 303.45,
   'pressure': 1016,
   'humidity': 62,
   'dew_point': 293.56,
   'clouds': 40,
   'visibility': 10000,
   'wind_speed': 5.14,
   'wind_deg': 250,
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}]}]}

In [10]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

# Assuming you have your 'completed_orders' DataFrame loaded
# (including 'Trip Start Time', 'Trip Origin_latitude', and 'Trip Origin_longitude')

# 1. Public Holiday Data (Example using a public API)
# Get public holidays for Nigeria (replace with appropriate API/data source if needed)
def get_public_holidays(year):
    url = f"https://date.nager.at/api/v3/PublicHolidays/{year}/NG"
    response = requests.get(url)
    if response.status_code == 200:
        return [datetime.strptime(holiday['date'], '%Y-%m-%d') for holiday in response.json()]
    else:
        return []

holidays_2021 = get_public_holidays(2021)  # Adjust years as needed for your data

df['is_holiday'] = df['Trip Start Time'].dt.date.isin(holidays_2021).astype(int)

# 2. Weekend vs. Weekday
df['is_weekend'] = df['Trip Start Time'].dt.dayofweek.isin([5, 6]).astype(int)

# 3. Hour of the Day (Already extracted in previous steps)

# 4. Weather Data (Example using OpenWeatherMap API)
# You'll need to get an API key from OpenWeatherMap
api_key = "YOUR_API_KEY" 

def get_weather(timestamp, lat, lon):
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "lat=" + str(lat) + "&lon=" + str(lon) + "&appid=" + api_key
    response = requests.get(complete_url) 
    x = response.json()
    if x["cod"] != "404":
        return x['weather'][0]['main']  # Simplify to just the main weather condition
    else:
        return np.nan  # Missing if API call fails

# Apply the weather function to get weather conditions (This might take a while for a large dataset)
df['weather_condition'] = df.apply(lambda row: get_weather(row['Trip Start Time'], row['Trip Origin_latitude'], row['Trip Origin_longitude']), axis=1)

# 5. Traffic Data (Example using TomTom API or other traffic data sources)
# You'll need to adapt this to your chosen API/data source.
# This is a complex process and may require additional data processing based on your API's response.

# 6. Special Occasions (This requires manual research or finding relevant datasets)
# You'll need to identify special events in Lagos for your data's time period and create a feature.


# 7. Relevance Analysis
# Example: Use a logistic regression model to assess feature importance
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Choose features and target variable
X = df[['is_holiday', 'is_weekend', 'hour_of_day', 'weather_condition']]  # Add more features as available
y = df['trip_duration'] # Or another target variable representing order completion

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

# Print feature importance
feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': model.coef_[0]})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)


ModuleNotFoundError: No module named 'requests'

#### Calucalting route information

In [6]:
import requests
import polyline
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime

# Replace 'YOUR_API_KEY' with your actual OpenRouteService API key

def calculate_route_info(start_coords, end_coords, start_time, end_time):
    """Calculates route information using OpenRouteService API."""

    url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={ORS_API_KEY}&start={start_coords[1]},{start_coords[0]}&end={end_coords[1]},{end_coords[0]}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # Extract key information from the API response
        total_distance = data['features'][0]['properties']['summary']['distance']
        total_duration = data['features'][0]['properties']['summary']['duration']  # In seconds
  # Corrected polyline decoding:
        decoded_polyline = polyline.decode(data['features'][0]['geometry']['coordinates'], geojson=True)
        
        # Calculate additional metrics
        straight_line_distance = geodesic(start_coords, end_coords).meters
        average_speed = (total_distance / total_duration) * 3.6  # Convert m/s to km/h
        trip_duration = end_time - start_time  # Assuming datetime objects

        return {
            'start_coords': start_coords,
            'end_coords': end_coords,
            'start_time': start_time,
            'end_time': end_time,
            'total_distance': total_distance,
            'total_duration': total_duration,
            'straight_line_distance': straight_line_distance,
            'average_speed': average_speed,
            'trip_duration': trip_duration.total_seconds(),
            'route_coordinates': decoded_polyline
        }
    else:
        print(f"Error: {response.status_code}")
        return None

# Example usage with the 'completed_orders' DataFrame
def enrich_completed_orders(df):
    results = []
    for _, row in df.iterrows():
        start_coords = (row['Trip Origin_latitude'], row['Trip Origin_longitude'])
        end_coords = (row['Trip Destination_latitude'], row['Trip Destination_longitude'])
        start_time = row['Trip Start Time']
        end_time = row['Trip End Time']
        
        route_info = calculate_route_info(start_coords, end_coords, start_time, end_time)
        if route_info:
            results.append(route_info)
    
    return pd.DataFrame(results)

# enrich your `completed_orders` DataFrame
enriched_df = enrich_completed_orders(df)
print(enriched_df.head().to_markdown(index=False, numalign="left", stralign="left"))


TypeError: ord() expected string of length 1, but list found